In [1]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import ast
from types import SimpleNamespace

In [3]:
# Import custom modules
from GNN_architectures import create_gnn_model
from GraphBuilder_with_features import GraphBuilder, create_graph_dataset
from training_utils import train

In [4]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [10]:
# Load data
def load_graph_data(loop):
    """Load graph data from CSV files."""
    edges = []
    y = []
    
    for i in range(loop, loop + 1):
        filename = f'../Graph_Edge_Data/den_graph_data_{loop}.csv'
        df = pd.read_csv(filename)
        edges += df['EDGES'].tolist()
        y += df['COEFFICIENTS'].tolist()
    
    edges = [ast.literal_eval(e) for e in edges]
    graphs_data = list(zip(edges, y))
    return graphs_data

In [9]:
# Define different feature configurations to test
def get_feature_configs():
    """Define different feature configurations for experiments."""
    return {
        'minimal': {
            'selected_features': ['basic', 'face'],
            'laplacian_pe_k': 0
        },
        'planar_focused': {
            'selected_features': ['basic', 'face', 'dual'],
            'laplacian_pe_k': 2
        },
        'balanced': {
            'selected_features': ['basic', 'face', 'spectral_node', 'centrality'],
            'laplacian_pe_k': 3
        },
        'full': {
            'selected_features': ['basic', 'face', 'spectral_node', 'dual', 'centrality'],
            'laplacian_pe_k': 4
        }
    }

In [6]:
# Main experiment configuration
def run_single_experiment(config,dataset ):
    """Run a single experiment with specified configuration."""

    config.in_channels = dataset[0].x.shape[1]
    
    print(f"Dataset created with {len(dataset)} graphs")
    print(f"Feature dimensions: {config.in_channels}")
    print(f"Feature names: {dataset[0].feature_names}")
    
    # Train model
    results = train(config, dataset)
    
    return results

# Experiments

In [7]:
# Load data
graphs_data = load_graph_data(loop=8)

In [11]:
# Standard Configurations 
get_feature_configs()

{'minimal': {'selected_features': ['basic', 'face'], 'laplacian_pe_k': 0},
 'planar_focused': {'selected_features': ['basic', 'face', 'dual'],
  'laplacian_pe_k': 2},
 'balanced': {'selected_features': ['basic',
   'face',
   'spectral_node',
   'centrality'],
  'laplacian_pe_k': 3},
 'full': {'selected_features': ['basic',
   'face',
   'spectral_node',
   'dual',
   'centrality'],
  'laplacian_pe_k': 4}}

In [12]:
# Dataset configuration
dataset_config = SimpleNamespace( selected_features=['basic', 'face', 'spectral_node', 'centrality'],
laplacian_pe_k=3) #Balanced configuration

# Create dataset
dataset, scaler = create_graph_dataset( graphs_data,
    {
        'selected_features': dataset_config.selected_features,
        'laplacian_pe_k': dataset_config.laplacian_pe_k
    }
)

Extracting features...
Normalizing features...
Created dataset with 1432 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']


In [ ]:
# Create configuration
config = SimpleNamespace(
    # Model configuration
    model_name='gin',
    hidden_channels=64,
    num_layers=3,
    dropout=0.2,
    
    # Training configuration
    lr=0.1,
    weight_decay=5e-4,
    epochs=70,
    batch_size=32,
    early_stop_patience=20,
    scheduler_type='onecycle',
    
    # WandB configuration
    use_wandb=True,
    project='gnn-planar-graphs',
    experiment_name='debugging_scheduler',
)

In [15]:
results = run_single_experiment(config, dataset )

Dataset created with 1432 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']
Using device: cpu


wandb: Currently logged in as: dian-gabriele (dian-gabriele-desydeutsches-elektronen-synchrotron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Starting training...
Epoch   0/70: Loss=0.6881, Acc=0.6432, Recall=0.6077, F1=0.6342
Epoch  10/70: Loss=0.7479, Acc=0.4749, Recall=0.6187, F1=0.5453
Epoch  20/70: Loss=0.7008, Acc=0.4979, Recall=0.5501, F1=0.5273
Epoch  30/70: Loss=0.6950, Acc=0.4979, Recall=0.5995, F1=0.5487
Epoch  40/70: Loss=0.6935, Acc=0.4839, Recall=0.4925, F1=0.4928
Epoch  50/70: Loss=0.6965, Acc=0.4825, Recall=0.4746, F1=0.4829
Epoch  60/70: Loss=0.6943, Acc=0.4700, Recall=0.1989, F1=0.2765
Epoch  69/70: Loss=0.6930, Acc=0.5091, Recall=1.0000, F1=0.6747


current_lr,▁▂▂▃▃▅▅▆▇▇█████████▇▇▇▇▆▆▆▅▅▅▄▄▂▂▂▂▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
train_accuracy,█▆▂▁▂▂▁▁▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▂▂
train_f1,▅▇██▇▇▄▄▃▅▄▃▄▃▃▁▂▃▄▂▂▃▄▂▂▃▄▃▄▂▃▅▂▄▄▆▆▆▆▆
train_loss,▆▁▁▂▆█▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
train_precision,▇▇█▇▄▂▂▂▃▂▂▃▂▃▂▂▂▂▂▂▂▂▃▃▂▂▃▂▃▂▂▂▂▁▃▂▂▂▂▂
train_recall,▅▆▆▇▇▅▅▅▄▆▅▄▄▄▅▂▆▃▄▄▅▃▄▄▃▄▄▅▃▅▅▃█▅▅█▁▇██
current_lr,0.0
epoch,69
train_accuracy,0.50908
train_f1,0.67469
